# Reference for ExtendedGravelBedrockEroder tests

This notebook shows the "ground truth" calculations for some of the unit tests used for the `ExtendedGravelBedrockEroder` component.

*(Greg Tucker, Susannah Morey, and Yuval Shmilovitz, March 2025)*

## Aspects to test

### "Instantaneous" tests

Tests should cover the following functionality:

1. Unlimited, homogeneous, fixed width, no attrition

2. Unlimited, homogeneous, fixed width, attrition

3. Unlimited, homogeneous, dynamic width, no attrition

4. Unlimited, multi-lithology, fixed width, attrition

5. Unlimited, multi-size, dynamic width, no attrition

6. Unlimited, multi-size, fixed width, no attrition

7. Unlimited, multi-size, fixed width, attrition

8. Limited, homogeneous, dynamic width, no attrition

9. Limited, multi-size, fixed width, no attrition

10. Limited, multi-size, fixed width, attrition

    


In [1]:
import numpy as np
from landlab import RasterModelGrid

#from landlab.components import FlowAccumulator, SoilGrading, GravelBedrockEroder

In [112]:
spy = 3600.0 * 24 * 365.25 # seconds per year

# Functions to use for test calculations
def slope(dz, dx):
    return dz / dx

def discharge(runoff_my, dx):
    area = dx * dx
    return runoff_my * area, runoff_my * area / spy

def fixed_width(kw, m, Q):
    return kw * Q**m

def rock_exposure_fraction(h, hc):
    return np.exp(-h / hc)

def median_grain_size(D):
    if type(D) is float:
        D50 = D
    else:
        D50 = np.median(D)
    return D50

def dynamic_width(rhos, rhow, eps, Q, S, D50, tau_star_c, g=9.81):
    R = (rhos - rhow) / rhow
    prefactor = 0.17 * g**(-0.5) * (R * (1 + eps) * tau_star_c)**(-5/3)
    return prefactor * Q * S**(7/6) / D50**1.5

def shear_stress(rho, g, rough, Qcms, w, S):
    return rho * g * rough**0.6 * (Qcms / w)**0.6 * S**0.7

def shields_stress(tau, rho, rhos, g, D50):
    return tau / ((rhos - rho) * g * D50)

def critical_shields(D, tau_st_c, mkomar):
    if type(D) is float:
        crit_sh = tau_st_c
    else:
        crit_sh = tau_st_c * (D / np.median(D))**mkomar
    return crit_sh
    
def critical_stress(tau_star_c_by_size, rho, rhos, g, D):
    sg = (rhos - rho) / rho
    tau_c = tau_star_c_by_size * sg * g * D
    
    return tau_c
    
def sed_capacity(intcy, mpm, w, rho, rhos, g, D50, tau_star, tau_star_c_by_size, f, mk):
    R = (rhos - rho) / rho
    exshields = np.maximum(tau_star - tau_star_c_by_size, 0.0)
    Qs_sec = f * intcy * mpm * w * (R * g)**0.5 * D50**1.5 * exshields**1.5
    Qs_yr = Qs_sec * spy
    return Qs_yr

def sed_flux(intcy, mpm, w, rho, rhos, g, D50, tau_star, tau_star_c_by_size, f, mk, br_exp_frac):
    cap = sed_capacity(
        intcy, mpm, w, rho, rhos, g, D50, tau_star, tau_star_c_by_size, f, mk
    )
    return (1.0 - br_exp_frac) * cap

def attrition_thickening_rate(sed_fluxes, beta, dx, cell_area):
    volume_loss_rate_per_length = sed_fluxes * beta
    volume_loss_rate = volume_loss_rate_per_length * dx
    thinning_rate = volume_loss_rate / cell_area
    return -thinning_rate

def outflux_thickening_rate(sed_fluxes, cell_area, porosity):
    return -sed_fluxes / ((1-porosity) *cell_area)

# def rock_plucking_rate(pluck_coef, shields, crit_shields_rock, width, dx, cellarea, br_frac):
#     exshields = max(shields - crit_shields_rock, 0.0)
#     pluck_rate = (
#         pluck_coef * (exshields)**1.5 
#         * width * dx / cellarea
#     )
#     return -br_frac * pluck_rate

def rock_plucking_rate(pluck_coef, tau, tau_crit_bedrock, tau_crit_grain, width, dx, cellarea, br_frac, intcy):

    if np.size(tau_crit_grain)>1:
        tau_crit_bedrock = np.ones_like(tau_crit_grain) * tau_crit_bedrock
        tau_crit_eff = np.max((tau_crit_bedrock, tau_crit_grain),0)
        exshields = tau - tau_crit_eff
        exshields[exshields<0]=0

    else:
        tau_crit_eff = np.max((tau_crit_bedrock, tau_crit_grain))
        exshields = np.max((tau - tau_crit_eff, 0.0))

    pluck_rate = (
        intcy * pluck_coef * (exshields)**1.5 
        * width * dx / cellarea
    )
    return -br_frac * pluck_rate


def rock_abrasion_rate(abr_coef, sedflux, chanlen, cellarea, brfrac):
    return -brfrac * abr_coef * sedflux * chanlen / cellarea

def set_default_params():
    p = {}
    p["sed0"] = 100.0              # initial sediment thickness, m
    p["phi"] = 0.5                 # Sediment porosity
    p["z0"] = 10.0                 # initial elevation, m
    p["dx"] = 1000.0               # grid cell width, m
    p["wid_coef"] = 0.001          # coef for width-vs-discharge (m and s units)
    p["wid_exp"] = 0.5             # exponent for width-vs-discharge
    p["runoff"] = 10.0             # bankfull runoff rate, m/y
    p["rho"] = 1000.0              # water density, kg/m3
    p["rhos"] = 2650.0             # sediment density, kg/m3
    p["n"] = 0.05                  # Manning roughness coef, s/m^1/3
    p["D"] = 0.01                  # median grain diameter, m
    p["tau_st_c"] = 0.0495         # critical Shields stress for median size
    p["g"] = 9.81                  # guess what?
    p["mpm"] = 3.97                # transport coefficient for MPM eqn
    p["intcy"] = 0.01              # intermittency factor
    p["cellarea"] = p["dx"]**2     # area of grid cell, m2
    p["beta"] = 0.0                # sed/rock abrasion coefficient, 1/m
    p["width_is_dynamic"] = False  # flag for dynamic vs fixed width
    p["epsilon"] = 0.2             # factor by which tau > tauc for dynamic wid
    p["num_classes"] = 1           # number of lithology or size classes
    p["class_fracs"] = 1.0         # proportion of each class in sediment
    p["mkomar"] = 0.68             # Komar's "m" exponent for critical shields stress
    p["hc"] = 1.0                  # characteristic thickness for bed cover, m
    p["kp"] = 1.0                  # plucking coefficient, m/y
    p["taustarcr"] = 0.0495        # critical shields stress for rock, FOR NOW: ASSUMING = tau_st_C
    p["br_abr_coef"] = 0.001       # bedrock abrasion coefficient, 1/m
    p["tau_crit_bedrock"] = 10     # Critical stress for bedrock plucking, pa

    return p

def calc_predicted_outputs(p):
    pv = {}
    pv["S"] = slope(p["z0"], p["dx"])
    pv["Q"], pv["Qcms"] = discharge(p["runoff"], p["dx"])
    pv["D50"] = median_grain_size(p["D"])
    if p["width_is_dynamic"]:
        pv["w"] = dynamic_width(
            p["rhos"], p["rho"], p["epsilon"], pv["Qcms"], pv["S"],
            pv["D50"], p["tau_st_c"], p["g"])
    else:
        pv["w"] = fixed_width(p["wid_coef"], p["wid_exp"], pv["Q"])
    pv["alpha"] = rock_exposure_fraction(p["sed0"], p["hc"])
    pv["tau"] = shear_stress(p["rho"], p["g"], p["n"], pv["Qcms"], pv["w"], pv["S"])
    pv["tau_star"] = shields_stress(pv["tau"], p["rho"], p["rhos"], p["g"], pv["D50"])
    pv["R"] = (p["rhos"] - p["rho"]) / p["rho"] # intermediate
    pv["crit_shields"] = critical_shields(
        p["D"], p["tau_st_c"], p["mkomar"]) 
    pv["Qs"] = sed_flux(
        p["intcy"], p["mpm"], pv["w"], p["rho"], p["rhos"], p["g"], pv["D50"],
        pv["tau_star"], pv["crit_shields"], p["class_fracs"], p["mkomar"],
        pv["alpha"]
    )
    if type(pv["Qs"]) is np.ndarray:
        pv["Qs_total"] = np.sum(pv["Qs"])
    else:
        pv["Qs_total"] = pv["Qs"]
    dHdt_outflux = outflux_thickening_rate(pv["Qs"], p["cellarea"], p["phi"])
    pv["dHdt_attr"] = attrition_thickening_rate(
        pv["Qs"], p["beta"], p["dx"], p["cellarea"]
    )
    
    pv["critical_stress"] = critical_stress(p["tau_st_c"], p["rho"], p["rhos"], p["g"], p["D"])

    pv["pluck_rate"] = rock_plucking_rate(p["kp"], pv["tau"], p["tau_crit_bedrock"], pv["critical_stress"],
                                pv["w"], p["dx"], p["cellarea"], pv["alpha"], p["intcy"])

    # pv["pluck_rate"] = rock_plucking_rate(
    #     p["kp"], pv["tau"], p["taustarcr"], pv["w"], p["dx"],
    #     p["cellarea"], pv["alpha"]
    # )

    pv["rock_abr_rate"] = rock_abrasion_rate(
        p["br_abr_coef"], pv["Qs_total"], p["dx"], p["cellarea"],
        pv["alpha"]
    )
    pv["dHdt"] = dHdt_outflux + pv["dHdt_attr"]
    if p["num_classes"] == 1:
        pv["dHdt_total"] = pv["dHdt"]
    else:
        pv["dHdt_total"] = np.sum(pv["dHdt"])
    pv["dzrdt"] = pv["pluck_rate"] + pv["rock_abr_rate"]
    pv["dzdt"] = pv["dHdt_total"] + pv["dzrdt"]

    return pv

def display_pred_vals(p, pv):
    print("PREDICTED VALUES:")
    print("slope", pv["S"])
    print("discharge (cmy)", pv["Q"])
    print("discharge (cms)", pv["Qcms"])
    print("channel width (m)", pv["w"])
    print("bedrock exposure fraction", pv["alpha"])
    print("bed shear stress (Pa)", pv["tau"])
    print("Shields stress", pv["tau_star"])
    print("Critical Shields stress", pv["crit_shields"])
    print("Critical Stress", pv["critical_stress"])
    print("sediment flux (cmy)", pv["Qs"])
    print("total sediment flux (cmy)", pv["Qs_total"])
    print("rate of sediment change from attrition (m/y)", pv["dHdt_attr"])
    print("rate of sediment thickness change by class (m/y)", pv["dHdt"])
    print("total rate of sediment thickness change (m/y)", pv["dHdt_total"])
    print("rate of rock plucking (m/y)", pv["pluck_rate"])
    print("rate of rock abrasion (m/y)", pv["rock_abr_rate"])
    print("rate of surface change (m/y)", pv["dzdt"])


## Basic 1-node instantaneous tests

Here we use a 3x3-node raster grid with a single core node and cell. The cell is 1 x 1 km, and the initial height is 10 m, with one open boundary, for a total slope of 1/100. The runoff rate is 10 m/y, for a discharge out of the cell of 10$^7$ cmy.

### Test 1: unlimited sediment, homogeneous, fixed width, no attrition

Default input parameters:

- $k_w = 0.001$ for $Q$ in cmy, and exponent = 1/2
- $D_{50} = 0.01$ m
- Initial sediment thickness 100 m
- $\tau_{*c} = 0.0495$
- $M = 3.97$
- $\rho = 1000$ kg/m$^3$
- $\rho_s = 2650$ kg/m$^3$


In [113]:
# Input parameters
p = set_default_params()

# Predicted outputs
pv = calc_predicted_outputs(p)

# Display them
display_pred_vals(p, pv)

PREDICTED VALUES:
slope 0.01
discharge (cmy) 10000000.0
discharge (cms) 0.3168808781402895
channel width (m) 3.1622776601683795
bedrock exposure fraction 3.72007597602e-44
bed shear stress (Pa) 16.277534696008008
Shields stress 0.10056241124398732
Critical Shields stress 0.0495
Critical Stress 0.008012317500000001
sediment flux (cmy) 183.91744631
total sediment flux (cmy) 183.91744631
rate of sediment change from attrition (m/y) -0.0
rate of sediment thickness change by class (m/y) -0.00036783489262
total rate of sediment thickness change (m/y) -0.00036783489262
rate of rock plucking (m/y) -1.85027164022e-47
rate of rock abrasion (m/y) -6.8418687359e-48
rate of surface change (m/y) -0.00036783489262


### Test 2: unlimited sediment, homogeneous, fixed width, attrition

- $\beta = 0.001$ 1/m


In [114]:
# Input parameters
p = set_default_params()
p["beta"] = 0.001

# Predicted outputs
pv = calc_predicted_outputs(p)

# Display them
display_pred_vals(p, pv)

PREDICTED VALUES:
slope 0.01
discharge (cmy) 10000000.0
discharge (cms) 0.3168808781402895
channel width (m) 3.1622776601683795
bedrock exposure fraction 3.72007597602e-44
bed shear stress (Pa) 16.277534696008008
Shields stress 0.10056241124398732
Critical Shields stress 0.0495
Critical Stress 0.008012317500000001
sediment flux (cmy) 183.91744631
total sediment flux (cmy) 183.91744631
rate of sediment change from attrition (m/y) -0.00018391744631
rate of sediment thickness change by class (m/y) -0.000551752338931
total rate of sediment thickness change (m/y) -0.000551752338931
rate of rock plucking (m/y) -1.85027164022e-47
rate of rock abrasion (m/y) -6.8418687359e-48
rate of surface change (m/y) -0.000551752338931


### Test 3: unlimited sediment, homogeneous, dynamic width, no attrition


In [115]:
# Input parameters
p = set_default_params()
p["width_is_dynamic"] = True

# Predicted outputs
pv = calc_predicted_outputs(p)

# Display them
display_pred_vals(p, pv)

PREDICTED VALUES:
slope 0.01
discharge (cmy) 10000000.0
discharge (cms) 0.3168808781402895
channel width (m) 3.8317144367318403
bedrock exposure fraction 3.72007597602e-44
bed shear stress (Pa) 14.506169527213025
Shields stress 0.08961893878981265
Critical Shields stress 0.0495
Critical Stress 0.008012317500000001
sediment flux (cmy) 155.198757487
total sediment flux (cmy) 155.198757487
rate of sediment change from attrition (m/y) -0.0
rate of sediment thickness change by class (m/y) -0.000310397514973
total rate of sediment thickness change (m/y) -0.000310397514973
rate of rock plucking (m/y) -1.36350342903e-47
rate of rock abrasion (m/y) -5.77351169235e-48
rate of surface change (m/y) -0.000310397514973


### Test 4: unlimited sediment, multi-lithology, fixed width, attrition

Input parameters:

- $\beta = 0.001, 0.0001$

In [116]:
# Input parameters
p = set_default_params()
p["num_classes"] = 2
p["beta"] = np.array([0.001, 0.0001])
p["class_fracs"] = np.array([0.5, 0.5])

# Predicted outputs
pv = calc_predicted_outputs(p)

# Display them
display_pred_vals(p, pv)


PREDICTED VALUES:
slope 0.01
discharge (cmy) 10000000.0
discharge (cms) 0.3168808781402895
channel width (m) 3.1622776601683795
bedrock exposure fraction 3.72007597602e-44
bed shear stress (Pa) 16.277534696008008
Shields stress 0.10056241124398732
Critical Shields stress 0.0495
Critical Stress 0.008012317500000001
sediment flux (cmy) [ 91.95872316  91.95872316]
total sediment flux (cmy) 183.91744631
rate of sediment change from attrition (m/y) [ -9.19587232e-05  -9.19587232e-06]
rate of sediment thickness change by class (m/y) [-0.00027588 -0.00019311]
total rate of sediment thickness change (m/y) -0.000468989488091
rate of rock plucking (m/y) -1.85027164022e-47
rate of rock abrasion (m/y) -6.8418687359e-48
rate of surface change (m/y) -0.000468989488091


### Test 5: Unlimited, multi-size, dynamic width, no attrition

In [117]:
# Input parameters
p = set_default_params()
p["num_classes"] = 5
p["class_fracs"] = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
p["D"] = np.array([0.001, 0.002, 0.004, 0.008, 0.016])
p["width_is_dynamic"] = True

# Predicted outputs
pv = calc_predicted_outputs(p)

# Display them
display_pred_vals(p, pv)

PREDICTED VALUES:
slope 0.01
discharge (cmy) 10000000.0
discharge (cms) 0.3168808781402895
channel width (m) 15.1461812043
bedrock exposure fraction 3.72007597602e-44
bed shear stress (Pa) 6.35926233068
Shields stress 0.0982186132066
Critical Shields stress [ 0.01928432  0.03089618  0.0495      0.07930592  0.12705916]
Critical Stress [ 0.00080123  0.00160246  0.00320493  0.00640985  0.01281971]
sediment flux (cmy) [ 85.66287378  67.47358354  41.53712723  10.04671818   0.        ]
total sediment flux (cmy) 204.720302724
rate of sediment change from attrition (m/y) [-0. -0. -0. -0. -0.]
rate of sediment thickness change by class (m/y) [ -1.71325748e-04  -1.34947167e-04  -8.30742545e-05  -2.00934364e-05
  -0.00000000e+00]
total rate of sediment thickness change (m/y) -0.000409440605449
rate of rock plucking (m/y) [-0. -0. -0. -0. -0.]
rate of rock abrasion (m/y) -7.61575079969e-48
rate of surface change (m/y) [-0.00040944 -0.00040944 -0.00040944 -0.00040944 -0.00040944]


### Test 6: Unlimited, multi-size, fixed width, no attrition

In [118]:
# Input parameters
p = set_default_params()
p["num_classes"] = 5
p["class_fracs"] = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
p["D"] = np.array([0.001, 0.002, 0.004, 0.008, 0.016])

# Predicted outputs
pv = calc_predicted_outputs(p)

# Display them
display_pred_vals(p, pv)

PREDICTED VALUES:
slope 0.01
discharge (cmy) 10000000.0
discharge (cms) 0.3168808781402895
channel width (m) 3.1622776601683795
bedrock exposure fraction 3.72007597602e-44
bed shear stress (Pa) 16.277534696008008
Shields stress 0.25140602811
Critical Shields stress [ 0.01928432  0.03089618  0.0495      0.07930592  0.12705916]
Critical Stress [ 0.00080123  0.00160246  0.00320493  0.00640985  0.01281971]
sediment flux (cmy) [ 90.19128187  83.50892907  73.16693464  57.57880604  35.36251861]
total sediment flux (cmy) 339.808470226
rate of sediment change from attrition (m/y) [-0. -0. -0. -0. -0.]
rate of sediment thickness change by class (m/y) [ -1.80382564e-04  -1.67017858e-04  -1.46333869e-04  -1.15157612e-04
  -7.07250372e-05]
total rate of sediment thickness change (m/y) -0.000679616940452
rate of rock plucking (m/y) [ -1.85027164e-47  -1.85027164e-47  -1.85027164e-47  -1.85027164e-47
  -1.85027164e-47]
rate of rock abrasion (m/y) -1.26411332654e-47
rate of surface change (m/y) [-0.00

### Test 7: Unlimited, multi-size, fixed width, attrition

In [119]:
# Input parameters
p = set_default_params()
p["num_classes"] = 5
p["class_fracs"] = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
p["D"] = np.array([0.001, 0.002, 0.004, 0.008, 0.016])
p["beta"] = 0.001

# Predicted outputs
pv = calc_predicted_outputs(p)

# Display them
display_pred_vals(p, pv)

PREDICTED VALUES:
slope 0.01
discharge (cmy) 10000000.0
discharge (cms) 0.3168808781402895
channel width (m) 3.1622776601683795
bedrock exposure fraction 3.72007597602e-44
bed shear stress (Pa) 16.277534696008008
Shields stress 0.25140602811
Critical Shields stress [ 0.01928432  0.03089618  0.0495      0.07930592  0.12705916]
Critical Stress [ 0.00080123  0.00160246  0.00320493  0.00640985  0.01281971]
sediment flux (cmy) [ 90.19128187  83.50892907  73.16693464  57.57880604  35.36251861]
total sediment flux (cmy) 339.808470226
rate of sediment change from attrition (m/y) [ -9.01912819e-05  -8.35089291e-05  -7.31669346e-05  -5.75788060e-05
  -3.53625186e-05]
rate of sediment thickness change by class (m/y) [-0.00027057 -0.00025053 -0.0002195  -0.00017274 -0.00010609]
total rate of sediment thickness change (m/y) -0.00101942541068
rate of rock plucking (m/y) [ -1.85027164e-47  -1.85027164e-47  -1.85027164e-47  -1.85027164e-47
  -1.85027164e-47]
rate of rock abrasion (m/y) -1.26411332654e

### Test 8: Limited, homogeneous, dynamic width, no attrition

In [120]:
# Input parameters
p = set_default_params()
p["sed0"] = 0.1
p["width_is_dynamic"] = True

# Predicted outputs
pv = calc_predicted_outputs(p)

# Display them
display_pred_vals(p, pv)

PREDICTED VALUES:
slope 0.01
discharge (cmy) 10000000.0
discharge (cms) 0.3168808781402895
channel width (m) 3.8317144367318403
bedrock exposure fraction 0.904837418036
bed shear stress (Pa) 14.506169527213025
Shields stress 0.08961893878981265
Critical Shields stress 0.0495
Critical Stress 0.008012317500000001
sediment flux (cmy) 14.76911448
total sediment flux (cmy) 14.76911448
rate of sediment change from attrition (m/y) -0.0
rate of sediment thickness change by class (m/y) -2.95382289601e-05
total rate of sediment thickness change (m/y) -2.95382289601e-05
rate of rock plucking (m/y) -0.000331646162649
rate of rock abrasion (m/y) -1.33636474128e-05
rate of surface change (m/y) -0.000374548039021


### Test 9: Limited, multi-size, fixed width, no attrition

In [121]:
# Input parameters
p = set_default_params()
p["sed0"] = 0.1
p["num_classes"] = 5
p["class_fracs"] = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
p["D"] = np.array([0.001, 0.002, 0.004, 0.008, 0.016])

# Predicted outputs
pv = calc_predicted_outputs(p)

# Display them
display_pred_vals(p, pv)

PREDICTED VALUES:
slope 0.01
discharge (cmy) 10000000.0
discharge (cms) 0.3168808781402895
channel width (m) 3.1622776601683795
bedrock exposure fraction 0.904837418036
bed shear stress (Pa) 16.277534696008008
Shields stress 0.25140602811
Critical Shields stress [ 0.01928432  0.03089618  0.0495      0.07930592  0.12705916]
Critical Stress [ 0.00080123  0.00160246  0.00320493  0.00640985  0.01281971]
sediment flux (cmy) [ 8.58283525  7.94692531  6.96275441  5.47934785  3.36518858]
total sediment flux (cmy) 32.3370514
rate of sediment change from attrition (m/y) [-0. -0. -0. -0. -0.]
rate of sediment thickness change by class (m/y) [ -1.71656705e-05  -1.58938506e-05  -1.39255088e-05  -1.09586957e-05
  -6.73037715e-06]
total rate of sediment thickness change (m/y) -6.46741027999e-05
rate of rock plucking (m/y) [-0.00045004 -0.00045004 -0.00045004 -0.00045004 -0.00045004]
rate of rock abrasion (m/y) -2.92597740956e-05
rate of surface change (m/y) [-0.00054398 -0.00054398 -0.00054398 -0.000

### Limited, multi-size, fixed width, attrition

In [122]:
# Input parameters
p = set_default_params()
p["sed0"] = 0.1
p["num_classes"] = 5
p["class_fracs"] = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
p["D"] = np.array([0.001, 0.002, 0.004, 0.008, 0.016])
p["beta"] = 0.001

# Predicted outputs
pv = calc_predicted_outputs(p)

# Display them
display_pred_vals(p, pv)

PREDICTED VALUES:
slope 0.01
discharge (cmy) 10000000.0
discharge (cms) 0.3168808781402895
channel width (m) 3.1622776601683795
bedrock exposure fraction 0.904837418036
bed shear stress (Pa) 16.277534696008008
Shields stress 0.25140602811
Critical Shields stress [ 0.01928432  0.03089618  0.0495      0.07930592  0.12705916]
Critical Stress [ 0.00080123  0.00160246  0.00320493  0.00640985  0.01281971]
sediment flux (cmy) [ 8.58283525  7.94692531  6.96275441  5.47934785  3.36518858]
total sediment flux (cmy) 32.3370514
rate of sediment change from attrition (m/y) [ -8.58283525e-06  -7.94692531e-06  -6.96275441e-06  -5.47934785e-06
  -3.36518858e-06]
rate of sediment thickness change by class (m/y) [ -2.57485058e-05  -2.38407759e-05  -2.08882632e-05  -1.64380435e-05
  -1.00955657e-05]
total rate of sediment thickness change (m/y) -9.70111541999e-05
rate of rock plucking (m/y) [-0.00045004 -0.00045004 -0.00045004 -0.00045004 -0.00045004]
rate of rock abrasion (m/y) -2.92597740956e-05
rate o